In [1]:
%load_ext autoreload
%autoreload 2

Current issue 3/4/25: Jubilee pressure cell doesn't have pump set up right, throwing errors on the AFL side of things. Integration otherwise seems to work OK

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
from science_jubilee.tools import PneumaticSampleLoader
import time
import numpy as np

import logging
import sys


In [3]:

import requests

In [4]:
r = requests.get('http://192.168.1.2/rr_connect')

In [5]:

r.content

b'<html>\n<head>\n</head>\n<body>\n<p style="font-size: 16pt; text-align: center; margin-top:50px">Your Duet rejected the HTTP request: Unknown request</p>\n</body>\n'

In [6]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response text in gcode:  {"key":"move.axes[].homed","flags":"","result":[true,true,true,true],"next":0}

response in connect:  [True, True, True, True]
response text in gcode:  {"key":"move.axes[]","flags":"","result":[{"acceleration":1500.0,"babystep":0,"current":3005,"drivers":["1.0"],"homed":true,"jerk":500.0,"letter":"X","machinePosition":6.000,"max":313.75,"maxProbed":false,"microstepping":{"interpolated":true,"value":16},"min":-13.75,"minProbed":false,"percentCurrent":100,"percentStstCurrent":75,"speed":18000.0,"stepsPerMm":200.00,"userPosition":5.000,"visible":true,"workplaceOffsets":[0,0,0,0,0,0,0,0,0]},{"acceleration":1500.0,"babystep":0,"current":3005,"drivers":["1.1"],"homed":true,"jerk":500.0,"letter":"Y","machinePosition":164.800,"max":341.00,"maxProbed":false,"microstepping":{"interpolated":true,"value":16},"min":-44.00,"minProbed":false,"percentCurrent":100,"percentStstCurrent":75,"speed":18000.0,"stepsPerMm":200.00,"userPosition":200.000,"visible":true,"workplaceOffse

In [7]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [8]:
safe_pos = (5, 200, 100)
cell_pos = (200, 50, 20)
url = 'http://192.168.1.4'
port = '5000'
name = 'PSL'
username = 'test'
password = 'domo_arigato'

In [9]:
jubilee.move_to(x = safe_pos[0], y = safe_pos[1], z = safe_pos[2])

response text in gcode:  

response text in gcode:  

response text in gcode:  



In [10]:
jubilee.

SyntaxError: invalid syntax (171782135.py, line 1)

In [11]:
psl = PneumaticSampleLoader.PneumaticSampleLoader(url, port, name, cell_pos, safe_pos, username, password)

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'


In [12]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")

In [13]:
jubilee.load_tool(syringe_10)
mix_syringe = syringe_10

response text in gcode:  {"key":"tools","flags":"","result":[null,{"active":[],"axes":[[0],[1]],"extruders":[],"fans":[0],"feedForward":[],"filamentExtruder":-1,"heaters":[],"isRetracted":false,"mix":[],"name":"Syringe_1","number":1,"offsets":[-2.000,37.900,-161.500,0],"offsetsProbed":0,"retraction":{"extraRestart":0,"length":2.0,"speed":16.7,"unretractSpeed":16.7,"zHop":0},"spindle":-1,"spindleRpm":0,"standby":[],"state":"standby"},{"active":[],"axes":[[0],[1]],"extruders":[],"fans":[0],"feedForward":[],"filamentExtruder":-1,"heaters":[],"isRetracted":false,"mix":[],"name":"Syringe_2","number":2,"offsets":[-1.000,35.200,-178.500,0],"offsetsProbed":0,"retraction":{"extraRestart":0,"length":2.0,"speed":16.7,"unretractSpeed":16.7,"zHop":0},"spindle":-1,"spindleRpm":0,"standby":[],"state":"active"},{"active":[],"axes":[[0],[1]],"extruders":[],"fans":[0],"feedForward":[],"filamentExtruder":-1,"heaters":[],"isRetracted":false,"mix":[],"name":"Syringe_3","number":3,"offsets":[-1.500,31.700,-

In [14]:
mix_syringe._machine

In [15]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
mix_syringe.set_pulsewidth(mix_syringe.full_position+250, s = 500)
mix_syringe.load_syringe(6200, mix_syringe.full_position+250)

Loaded syringe, remaining volume 0 uL
Loaded syringe, remaining volume 6200 uL


In [16]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [17]:
psl.arm_state

'UP'

In [18]:
psl.get_cell_state()

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'


'READY'

In [19]:
jubilee.load_tool(psl)

In [ ]:
psl.rinse_cell()

In [24]:
psl.unpause_queue()

In [ ]:
psl.load_sample(mix_syringe, samples['A1'], 500)

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': True, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': True}","Rinse Program Step 0/6: rinse1 for 5s","No action yet..."]\n'
status r co

In [35]:
jubilee.tools.values()

dict_values([{'name': '10cc_1', 'tool': <science_jubilee.tools.HTTPSyringe.HTTPSyringe object at 0x7f07880784d0>}, {'name': 'PSL', 'tool': <science_jubilee.tools.PneumaticSampleLoader.PneumaticSampleLoader object at 0x7f07880f3810>}])

In [37]:
mix_syringe.index

2

In [ ]:
jubilee._get_tool_index(jubilee.active_tool_index

In [42]:
jubilee.park_tool()

response text in gcode:  X:10.000 Y:200.000 Z:200.000 U:0.000 E:0.000 Count 42000 -38000 320000 0 Machine 10.000 200.000 200.000 0.000 Bed comp 0.000

response text in gcode:  

response text in gcode:  No tool is selected



In [41]:
jubilee.active_tool_index

response text in gcode:  No tool is selected



-1

In [20]:
print(jubilee)

In [26]:
jubilee.session